# Char-RNN stateless
* This RNN generate char by char sentences trained by Shakespeare's work
* this RNN is stateless, so the state at eache timestep is set to zero

## Import dependencies

In [82]:
import numpy as np

import tensorflow as tf
from tensorflow import keras

## Creating Dataset

In [6]:
#Download data
shakespeare_url = "http://homl.info/shakespeare"
filepath = keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as file:
    shakespeare_text = file.read()

1122304/1115394 [==============================] - 9s 8us/step


In [73]:
# Tokenize chars
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)  # char_level=True allows char 
                                                                 # encoding instead default word enoding
tokenizer.fit_on_texts(shakespeare_text)

In [74]:
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1  # [encoded] = encoded.squeeze()

In [83]:
# Split Sequential dataset
dataset_size = tokenizer.document_count
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [87]:
# Chopping dataset into multiple windows
n_steps = 100
windows_length = n_steps + 1
dataset = dataset.window(windows_length, shift=1, drop_remainder=True)

In [89]:
dataset = dataset.flat_map(lambda window: window.batch(windows_length))

In [91]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:,:-1], windows[:, 1:]))

In [95]:
max_id = len(tokenizer.word_index)
dataset = dataset.map(lambda X_batch, y_batch: (tf.one_hot(X_batch, depth=max_id), y_batch))
dataset = dataset.prefetch(1)

## Building and training the model

In [100]:
model = keras.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id], dropout=.2, recurrent_dropout=.2),
    keras.layers.GRU(128, return_sequences=True, dropout=.2, recurrent_dropout=.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation="softmax"))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, epochs=20)

Epoch 1/20
31368/31368 [==============================] - 10119s 323ms/step - loss: 1.4665
Epoch 2/20
31368/31368 [==============================] - 10300s 328ms/step - loss: 1.3626
Epoch 3/20
31368/31368 [==============================] - 6970s 222ms/step - loss: 1.3425
Epoch 4/20
31368/31368 [==============================] - 6831s 218ms/step - loss: 1.3321
Epoch 5/20
31368/31368 [==============================] - 6341s 202ms/step - loss: 1.3250
Epoch 6/20
31368/31368 [==============================] - 6367s 203ms/step - loss: 1.3200
Epoch 7/20
31368/31368 [==============================] - 6357s 203ms/step - loss: 1.3165
Epoch 8/20
31368/31368 [==============================] - 6388s 204ms/step - loss: 1.3137
Epoch 9/20
31368/31368 [==============================] - 10469s 334ms/step - loss: 1.3111
Epoch 10/20
28459/31368 [==========================>...] - ETA: 10:04 - loss: 1.3112

ResourceExhaustedError:  OOM when allocating tensor with shape[100,32,128] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[node sequential_1/gru_7/TensorArrayV2Stack/TensorListStack (defined at <ipython-input-100-235f4af9fd06>:7) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_7443]

Function call stack:
train_function


## Predict with the model

In [117]:
def process(text):
    X = np.array(tokenizer.texts_to_sequences(text)) - 1
    return tf.one_hot(X, max_id)

In [121]:
def next_char(text, temperature=1):
    X_new = process([text])
    y_proba = model.predict(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [124]:
def complete_text(text, n_char=50, temperature=1):
    for _ in range(n_char):
        text += next_char(text, temperature)
    return text

In [130]:
print(complete_text("hello, my name is daniel I am the best ", temperature=.8))

hello, my name is daniel I am the best bear
the great will require to steal up it,
the si
